In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

In [2]:
df = pd.read_csv('t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

C:\Users\anees\AppData\Local\Temp\ipykernel_22344\3748546470.py:1: DtypeWarning: Columns (54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])


In [4]:
df.columns

Index(['p_match', 'inns', 'bat', 'p_bat', 'team_bat', 'bowl', 'p_bowl',
       'team_bowl', 'ball', 'ball_id', 'outcome', 'score', 'out', 'dismissal',
       'p_out', 'over', 'ball.1', 'noball', 'wide', 'byes', 'legbyes',
       'cur_bat_runs', 'cur_bat_bf', 'cur_bowl_ovr', 'cur_bowl_wkts',
       'cur_bowl_runs', 'inns_runs', 'inns_wkts', 'inns_balls',
       'inns_runs_rem', 'inns_balls_rem', 'inns_rr', 'inns_rrr', 'target',
       'max_balls', 'date', 'year', 'ground', 'country', 'winner', 'toss',
       'competition', 'bat_hand', 'bowl_style', 'bowl_kind', 'batruns',
       'ballfaced', 'bowlruns', 'bat_out', 'wagonX', 'wagonY', 'wagonZone',
       'pitchLine', 'pitchLength', 'shotType'],
      dtype='object')

In [5]:
df.head()

,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,batruns,ballfaced,bowlruns,bat_out,wagonX,wagonY,wagonZone,pitchLine,pitchLength,shotType
0,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,1,1001349_1_1_1,...,0,1,0,True,208,178,3,ON_THE_STUMPS,SHORT_OF_A_GOOD_LENGTH,DEFENDED
1,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,2,1001349_1_1_2,...,0,1,0,True,243,175,3,ON_THE_STUMPS,GOOD_LENGTH,DEFENDED
2,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,3,1001349_1_1_3,...,1,1,1,True,189,242,4,ON_THE_STUMPS,SHORT_OF_A_GOOD_LENGTH,DEFENDED
3,1001351,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,3,1001351_1_1_3,...,0,1,0,True,166,194,5,OUTSIDE_OFFSTUMP,SHORT_OF_A_GOOD_LENGTH,DEFENDED
4,1001351,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,4,1001351_1_1_4,...,4,1,4,True,15,222,6,OUTSIDE_OFFSTUMP,GOOD_LENGTH,CUT_SHOT


In [6]:
df.shape

(1555016, 55)

In [8]:
df['bowl_style'].unique()

array(['RF', 'RFM', 'LB', 'LWS', 'RMF', 'SLA', 'OB', 'LBG', 'LFM', 'LF',
       'RM', 'LMF', 'RM/OB', 'LM', 'LFM/SLA', 'OB/LB', 'LM/SLA/LWS',
       'OB/SLA', 'RAB', 'LS', '-', 'LSM', 'RSM', 'RMF/OB', 'RFM/OB',
       'OB/LBG', 'RFM/LBG', 'RM/LB', 'RM/LBG', 'LMF/RM', 'RM/RSM',
       'SLA/LWS', 'LAB', 'RMF/LB', 'RFM/LB', 'RM/OB/LB', 'RS'],
      dtype=object)

In [31]:
unique_styles = ['RFM', 'RMF', 'LBG', 'LB', 'LS', 'RAB', 'LAB', 'OB', 'SLA', 'RAB']
result = pd.concat([df[df['bowl_style'] == style][['bowl','bowl_style']].drop_duplicates().head(2) for style in unique_styles])
result

,bowl,bowl_style
2276,Nuwan Kulasekara,RFM
3078,Vikum Sanjaya,RFM
7715,Asela Gunaratne,RMF
28810,Andrew Tye,RMF
20353,Fawad Ahmed,LBG
40125,Adam Zampa,LBG
3598,Seekkuge Prasanna,LB
55231,Cameron Boyce,LB
715323,Mohammed Aslam,LS
1100250,Manson Chikowero,LS


There are few bowling types which have ambiguous values. So after researching the bowlers, we came to know:
1. RFM, RMF means Right Arm Medium Fast
2. LBG, LB means Leg Break
3. LS, SLA means Slow Left Arm Offie
4. OB means Offie
5. RAB,LAB means Right/Left Arm Bowler, who is likely a part timer who don't have any particular style
Now we are changing these values into more understandable format.

In [26]:
df[df['bowl_style'].isin(['RAB'])][['bowl','bowl_style']].drop_duplicates().head(50)

,bowl,bowl_style
703480,Madushanka Ekanayake,RAB
1102003,Sajjad Ali,RAB
1190282,Jonathan Barry,RAB


In [20]:
df[df['bowl_style']=='LB'][['bowl']].drop_duplicates()

,bowl
3598,Seekkuge Prasanna
55231,Cameron Boyce
58048,Liam O'Connor
64680,Mitchell Swepson
65930,Marnus Labuschagne
...,...
1524234,Khrievitso Kense
1525617,Amit Ali
1528022,Rosiamliana Ralte
1536364,Harsha Bharadwaj


In [51]:
df[(df['bat']=='Aryaveer Chaudhary') & (df['bowl_style']=='LMF')]

,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,winner,toss,competition,bat_hand,bowl_style,bowl_kind,batruns,ballfaced,bowlruns,bat_out
1535127,1405317,2,Aryaveer Chaudhary,1329997,Singapore,Bilal Khan,937953,Oman,2,17.02,...,Oman,Singapore,T20I,LHB,LMF,pace bowler,-4,1,-3,True
1535128,1405317,2,Aryaveer Chaudhary,1329997,Singapore,Bilal Khan,937953,Oman,3,17.03,...,Oman,Singapore,T20I,LHB,LMF,pace bowler,0,1,0,True
1535129,1405317,2,Aryaveer Chaudhary,1329997,Singapore,Bilal Khan,937953,Oman,6,17.06,...,Oman,Singapore,T20I,LHB,LMF,pace bowler,0,1,0,True
1535130,1405317,2,Aryaveer Chaudhary,1329997,Singapore,Bilal Khan,937953,Oman,7,17.07,...,Oman,Singapore,T20I,LHB,LMF,pace bowler,0,1,0,True
1535131,1405317,2,Aryaveer Chaudhary,1329997,Singapore,Bilal Khan,937953,Oman,3,19.03,...,Oman,Singapore,T20I,LHB,LMF,pace bowler,0,1,0,True
1535132,1405317,2,Aryaveer Chaudhary,1329997,Singapore,Bilal Khan,937953,Oman,4,19.04,...,Oman,Singapore,T20I,LHB,LMF,pace bowler,0,1,0,True


In [47]:
subset_df=df.groupby(['p_bat','bat','bowl_style'])['batruns'].sum().reset_index()

In [48]:
subset_df.sort_values(by='batruns',ascending=False)

,p_bat,bat,bowl_style,batruns
6529,249866,Alex Hales,RFM,1828
5908,232359,Colin Munro,RFM,1528
9168,308967,Jos Buttler,RF,1512
148,5334,Aaron Finch,RFM,1501
7956,296597,James Vince,RFM,1467
...,...,...,...,...
7206,275936,Ryan Wiggins,LWS,0
35716,1136011,Manik Khan,LFM,0
33457,1079842,Manjot Kalra,RM,0
36757,1155506,Shelton Nhavotso,LM,-2


In [1]:
df.shape()

NameError: name 'df' is not defined